In [1]:
import cohere
import os
import time
import re
import pickle
import numpy as np
import json

In [2]:
# Ścieżka do folderu z plikami
dataset_path = "../../ML_Model/datasets/tested_dataset/final_dataset/final_test.pkl"

# Preprocessing dataset

In [3]:
# Wczytanie pliku
with open(dataset_path, 'rb') as f:
    data = pickle.load(f)

In [4]:
labels = data['labels']
answers = data['answer']
evaluations = data['rating']
fps = data['fps']
filenames = data['video_url']
tricks = data['tricks']
labels_type = [
    "Squat-depth" ,
    "Back-round" ,
    "Taking-off-foot",
    "Knee-collapse" ,
    "Butt-wink" ,
    "Dominant-hip" ,
    "Lack-of-abdominal-tension",
    "No-knee-outlet" 
]

# Api

In [5]:
# Inicjalizacja klienta Cohere
# https://cohere.com/
# https://docs.cohere.com/reference/embed

with open('../api.txt', 'r', encoding='utf-8') as file:
    for line in file:
        if 'cohere:' in line:
            # Wyciągamy tekst po "cohere:"
            key_api = line.split('cohere:')[1].strip()
            
co = cohere.Client(key_api)

# Generowanie odpowiedzi

In [25]:
# Funkcja do generowania analizy techniki z komentarzy
def generate_technique_analysis_prompt(data, prompt):
    
    prompt += "Data: " + data[0]
    #prompt += f"\n Summary of technique:  {data[1]}"
    prompt += f"\n Overall evaluation:  {data[2]}"
    #prompt += f"\n fps of video: {data[3]}"
    
    response = co.generate(
        model='command',  # Poprawiony model
        prompt=prompt,
        max_tokens=300,
        temperature=0.7
    )
    print("Prompt: ", prompt)
    # Zwracamy wygenerowaną odpowiedź
    return response.generations[0].text

In [26]:
# Zbudowanie promptu na podstawie komentarzy
prompt = "You will get data about a recording in which there is a person squatting with a barbell and weight. Then you will get data for this recording, the first is an array where in each row there is an error and information about the time intervals in which the error occurred. Analyze this data and describe it for the user. Do not describe when the errors occurred, but rather what errors occurred and how many times. In the second section describe how the problems can be solved. List possible causes of their occurrence. Do not exceed 1000 characters. Avoid general answers like: 'The error is due to poor technique' "

In [27]:
analysis_comments_prompt = {}
# Ustawienie wyświetlania pełnych tablic
np.set_printoptions(threshold=np.inf)

# Wyświetlenie wyników
for i, filename in enumerate(filenames):
    start_time = time.time()

    print(f"Wyniki dla nagrania: {filename}, numer {i}")
    if i >= 10:
        break
        
    label = labels[i]
    answer = answers[i]
    evaluation = json.loads(evaluations[i])[0]["number"]
    fps_ = fps[i]
    trick = tricks[i]
    
    ## Łączenie labels z labels_type
    #combined_data = []
    #for label_row, label_type in zip(trick, labels_type):
    #    combined_data.append({"type": label_type, "values": label_row})
        
    result_string = ""
    if not isinstance(trick, float):
        for item in trick:
            result_string += f"start: {round(item['start'],2)}, end: {round(item['end'],2)} : {item['labels']}\n"
            
    data_input = [result_string, answer, evaluation, fps_]
    
    # Generowanie analizy techniki, jeśli komentarze istnieją
    analysis = generate_technique_analysis_prompt(data_input, prompt)

    
    # Dodajemy wynik do listy
    analysis_comments_prompt[filename] = {
        'Analysis': analysis
    }
    
    print('*'*100)
    print(f"Analysis: \n {analysis}\n")
    print('*'*100)
    
    end_time = time.time()
    print(f"Przetwarzanie trwało {end_time - start_time:.4f} sekundy")
    
    print("-" * 40)

Wyniki dla nagrania: s3://squat-label-studio/squats/lifting/0-0084-50-7-0-2-0-0.mp4, numer 0
Prompt:  You will get data about a recording in which there is a person squatting with a barbell and weight. Then you will get data for this recording, the first is an array where in each row there is an error and information about the time intervals in which the error occurred. Analyze this data and describe it for the user. Do not describe when the errors occurred, but rather what errors occurred and how many times. In the second section describe how the problems can be solved. List possible causes of their occurrence. Do not exceed 1000 characters. Avoid general answers like: 'The error is due to poor technique' Data: start: 13.37, end: 15.34 : ['Dominant-hip']
start: 25.8, end: 27.31 : ['Dominant-hip']

 Overall evaluation:  7
****************************************************************************************************
Analysis: 
  Analysis of the recording and description of the err

In [14]:
data

,annotation_id,annotator,answer,created_at,id,lead_time,rating,squat_types,tricks,updated_at,video_url,fps,video_time,frames,labels
0,13,1,NaN,2024-11-18T22:25:23.511525Z,366,698.367,"[{""number"":7}]",diffrent,"[{'start': 13.374851720047449, 'end': 15.34460...",2024-11-18T22:25:23.511525Z,s3://squat-label-studio/squats/lifting/0-0084-...,30.0,41.00,1230.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,11,1,Poprawny,2024-11-18T22:08:33.297645Z,367,267.864,"[{""number"":7}]",NaN,NaN,2024-11-18T22:09:00.052430Z,s3://squat-label-studio/squats/lifting/0-0085-...,30.0,14.00,420.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,12,1,Przysiad wykonywany jest zdecydowanie za szybk...,2024-11-18T22:15:06.188387Z,368,360.965,"[{""number"":6}]",NaN,"[{'start': 1.0688018979833926, 'end': 1.804863...",2024-11-18T22:15:06.188387Z,s3://squat-label-studio/squats/lifting/0-0086-...,30.0,17.00,510.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,14,1,"Przy każdym powtórzeniu zapadają się kolana, e...",2024-11-18T22:43:52.207631Z,379,295.713,"[{""number"":3}]",low_bar,"[{'start': 1.1694386682692304, 'end': 4.071379...",2024-11-18T22:43:52.207631Z,s3://squat-label-studio/squats/self_recorded/J...,29.0,27.03,810.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,15,1,"Przy każdym powtórzeniu zapadają się kolana, e...",2024-11-18T22:48:58.399091Z,380,304.659,"[{""number"":2}]",low_bar,"[{'start': 2.976266862179487, 'end': 4.6734768...",2024-11-18T22:48:58.399091Z,s3://squat-label-studio/squats/self_recorded/J...,29.0,23.02,689.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
5,16,1,"Przy każdym powtórzeniu zapadają się kolana, e...",2024-11-18T22:53:18.168373Z,381,258.320,"[{""number"":3}]",low_bar,"[{'start': 4.599685150641026, 'end': 6.4813745...",2024-11-18T22:53:18.168373Z,s3://squat-label-studio/squats/self_recorded/J...,29.0,23.02,689.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
6,17,1,Przysiad wykonywany zbyt płytko. Wszystkie pow...,2024-11-18T22:58:42.870782Z,382,321.282,"[{""number"":4}]",low_bar,"[{'start': 1.0181121282051284, 'end': 4.102392...",2024-11-18T22:58:42.870782Z,s3://squat-label-studio/squats/self_recorded/J...,29.0,28.03,840.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
7,18,1,Przysiad wykonywany zbyt płytko. Wszystkie pow...,2024-11-18T23:01:56.504754Z,383,190.570,"[{""number"":4}]",low_bar,"[{'start': 1.1437719759615383, 'end': 3.652691...",2024-11-18T23:01:56.504754Z,s3://squat-label-studio/squats/self_recorded/J...,29.0,23.02,689.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
8,19,1,Przysiad wykonywany zbyt płytko. Wszystkie pow...,2024-11-18T23:05:16.449992Z,384,184.997,"[{""number"":4}]",low_bar,"[{'start': 1.8201962628205124, 'end': 3.947858...",2024-11-18T23:05:16.449992Z,s3://squat-label-studio/squats/self_recorded/J...,29.0,23.02,689.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
9,20,1,Przy każdym potwórzeniu powtarza się zawijanie...,2024-11-18T23:08:22.312132Z,385,182.011,"[{""number"":4}]",low_bar,"[{'start': 2.156002153846154, 'end': 3.2340032...",2024-11-18T23:08:22.312132Z,s3://squat-label-studio/squats/self_recorded/J...,29.0,24.03,720.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
